In [2]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras.layers import *

from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.models import Sequential

import pathlib

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.layers import Concatenate
# Create the base model from the pre-trained model MobileNet V2


IMG_SIZE = (224,224)
IMG_SHAPE = IMG_SIZE + (3,)

data_dir = pathlib.Path('./Image data')
images=list(data_dir.glob('*/*.tif'))
image_dic= {
    'normal' : list(data_dir.glob("e0_Normal/*")),
    'hole':list(data_dir.glob("e1_hole/*")),
    'stain' : list(data_dir.glob("e2_Stain/*")),
    'net' :list(data_dir.glob("e3_Net/*")),
    'color' :list(data_dir.glob("e4_colour/*")),
    'crease': list(data_dir.glob("e5_Crease/*")),
}

labels = {
    'normal':0,
    'hole':1,
    'stain':2,
    'net':3,
    'color':4,
    'crease':5,
}
X,y=[],[]

for name, images in image_dic.items():
    for image in images:
        img = cv2.imread(str(image))
        
        
        if img is None:
            print('Wrong path:', image)
        else:
            resized = cv2.resize(img,IMG_SIZE)
            X.append(resized)
            y.append(labels[name])
X = np.array(X)/255
y= np.array(y)
X_train, X_test, y_train,y_test = train_test_split(X,y,random_state=0, shuffle=True,test_size=0.2)


Wrong path: Image data/e0_Normal/test1.tif.gz
Wrong path: Image data/e0_Normal/test.tif.gz
Wrong path: Image data/e0_Normal/test2.gif.gz


In [27]:
def Xception_block(input_shape, classes,
                   activation='sigmoid',
                   flows=7, 
                   ensemble='S16,3,2_R_D_S32,3,2_R_D',
                   loss='binary_crossentropy',
                   learning_rate=0.001,
                   weights='imagenet'):

    '''
    input_shape : input shape of image
    classes     : number of classes 
    breakindex  : number of xception block-flows to be used 
    ensemble    : the default ensemble model to be followed for each class
    '''

    assert flows > 0 and flows < 10, "The number of flows should be between [1-9]"
    assert loss in ['binary_crossentropy', 'categorical_crossentropy'], "loss can be either 'binary_crossentropy' or 'categorical_crossentropy'"

    tf.keras.backend.clear_session()
    breakindex = [36, 46, 56, 66, 76, 86, 96, 106, 116]
    
    input_img = tf.keras.Input(input_shape, name='input')
    base_model = tf.keras.applications.Xception(input_tensor=input_img, 
                                                include_top=False, 
                                                weights=weights)
    
    totLayers = len(base_model.layers)
    for i in range(totLayers):
        base_model.layers[i]._name += '_head'

    # Last index of the head CNN
    comb_out = base_model.layers[breakindex[flows-1]].output
    #base_model.layers[breakindex[flows-1]]._name += '_lastcom'

    seps = []
    if ensemble != '':
        esets = ensemble.split('_')
    else:
        esets = None

def parser(tok, id, cls):
    ''' 
    SepConv: S,f,k,s
    MaxPool: P,s
    ReLU   : R
    Dropout: D
    '''
    if tok[0] == 'S':
        filters, ks, stride = list(map(int, tok[1:].split(',')))
        return SeparableConv2D(filters=filters, kernel_size=ks, strides=stride, 
                                padding='same', name=f'sepconv2d_d{id}_c{cls}')
    elif tok[0] == 'P':
        return MaxPool2D(pool_size=int(tok[1:]), name=f'MaxPool2D_d{id}_c{cls}')
    elif tok[0] == 'R':
        return ReLU(name=f'ReLU_d{id}_c{cls}')
    elif tok[0] == 'D':
        return Dropout(0.1, name=f'dropout_d{id}_c{cls}')
    elif tok[0] == 'N':
        return BatchNormalization()
    else:
        print('Invalid')


# How to define ensemble structure:
# S: SeparableConv2D, follows by filter, kernel, and stride 
#    example: S64,3,2
# P: MaxPool, followed by pool size
#    example: P2
# R: ReLU activation
# D: Dropout, defaults to 0.1 drop
#
# A full example: 'S8,3,2_R_D_S8,3,2_R_D'
# The network is as follows:
#    SeparableConv2D(filters=8, kernel_size=3, strides=2)
#    ReLU()
#    Dropout(0.1)
#    SeparableConv2D(filters=8, kernel_size=3, strides=2)
#    ReLU()
#    Dropout(0.1)


ensemble_structure = ['S64,3,2_R_D_S64,3,2_R_D',
                      'S32,3,2_R_D_S64,3,2_R_D',
                      'S16,3,2_R_D_S32,3,2_R_D', 
                      'S8,3,2_R_D_S8,3,2_R_D',
                      'S4,3,2_S16,3,2']


def Xception_block(input_shape, classes,
                   activation='sigmoid',
                   flows=7, 
                   
                   loss='binary_crossentropy',
                   learning_rate=0.001,
                   weights='imagenet'):

    '''
    input_shape : input shape of image
    classes     : number of classes 
    breakindex  : number of xception block-flows to be used 
    ensemble    : the default ensemble model to be followed for each class
    '''

    assert flows > 0 and flows < 10, "The number of flows should be between [1-9]"
    assert loss in ['binary_crossentropy', 'categorical_crossentropy'], "loss can be either 'binary_crossentropy' or 'categorical_crossentropy'"

    tf.keras.backend.clear_session()
    breakindex = [6,16,26,36, 46, 56, 66, 76, 86, 96, 106, 116]
    
    input_img = tf.keras.Input(input_shape, name='input')
    base_model = tf.keras.applications.Xception(input_tensor=input_img, 
                                                include_top=False, 
                                                weights=weights)
    
    totLayers = len(base_model.layers)
    for i in range(totLayers):
        base_model.layers[i]._name += '_head'

    # Last index of the head CNN
    comb_out = base_model.layers[breakindex[flows-1]].output
    #base_model.layers[breakindex[flows-1]]._name += '_lastcom'

    seps = []
    

    for seg in range(classes):

        res = SeparableConv2D(filters=16, kernel_size=3, 
                                strides=2, name=f'sepconv2d_d1_c{seg}')(comb_out)
        res = ReLU(name=f'maxpool2d_d1_c{seg}')(res)
        res = SeparableConv2D(filters=8, kernel_size=3, strides=2, name=f'sepconv2d_d2_c{seg}')(res)
        res = ReLU(name=f'maxpool2d_d2_c{seg}')(res)

        res = Flatten(name='flatten_'+str(seg))(res)
        res = Dense(1)(res)
        seps.append(res)
        
    out = Concatenate()(seps)
    out = Activation('sigmoid' if loss=='binary_crossentropy' else 'softmax')(out)    
    model = tf.keras.Model(input_img, out)

    print('Params', model.count_params())

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
                  loss=loss,
                  metrics=[tf.keras.metrics.CategoricalAccuracy(),
                           tf.keras.metrics.BinaryAccuracy(),
                           tf.keras.metrics.Precision(),
                           tf.keras.metrics.Recall(),
                           tf.keras.metrics.TopKCategoricalAccuracy(),
                           tf.keras.metrics.AUC(num_thresholds=200,
                                                multi_label=True),
                           tf.keras.metrics.TruePositives(),
                           tf.keras.metrics.FalsePositives(),
                           ],
                  )
    return model

In [44]:
model = Xception_block(X_train.shape[1:], 6,
                       flows=6,
                       loss='binary_crossentropy')

for layer in model.layers[:25]:
    layer.trainable = False
    

Params 235222


In [45]:

history = model.fit(X_train,to_categorical(y_train) , validation_data=(X_test, to_categorical(y_test)), 
                    epochs=10,)

Epoch 1/10


2023-02-13 22:46:30.852602: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


31/31 [==============================] - ETA: 0s - loss: 0.5426 - categorical_accuracy: 0.1651 - binary_accuracy: 0.8024 - precision: 0.1745 - recall: 0.0498 - top_k_categorical_accuracy: 0.8629 - auc: 0.4998 - true_positives: 48.0000 - false_positives: 227.0000

2023-02-13 22:47:08.582044: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


31/31 [==============================] - 46s 1s/step - loss: 0.5426 - categorical_accuracy: 0.1651 - binary_accuracy: 0.8024 - precision: 0.1745 - recall: 0.0498 - top_k_categorical_accuracy: 0.8629 - auc: 0.4998 - true_positives: 48.0000 - false_positives: 227.0000 - val_loss: 0.4809 - val_categorical_accuracy: 0.1618 - val_binary_accuracy: 0.8333 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_top_k_categorical_accuracy: 0.8050 - val_auc: 0.4979 - val_true_positives: 0.0000e+00 - val_false_positives: 0.0000e+00
Epoch 2/10
31/31 [==============================] - 32s 1s/step - loss: 0.4508 - categorical_accuracy: 0.2118 - binary_accuracy: 0.8333 - precision: 0.0000e+00 - recall: 0.0000e+00 - top_k_categorical_accuracy: 0.8598 - auc: 0.5338 - true_positives: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: 0.4686 - val_categorical_accuracy: 0.1826 - val_binary_accuracy: 0.8333 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_top_k_categorical_accuracy: 0.8465 -